In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


텐서프로우로 단어 품사 구분하기  
'I work at google', 'I google at work'의 단어 품사를 구분하는 RNN 코드를 텐서플로우로 구현한다.  
각 단어는 원-핫 인코딩으로 표현한다.

In [2]:
# I      => [1, 0, 0, 0] # 대명사
# work   => [0, 1, 0, 0] # 동사
# at     => [0, 0, 1, 0] # 전치사
# google => [0, 0, 0, 1] # 명사

# I work at google => [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
# I google at work => [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]]

inputs = np.array([
    [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]],
    [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]]
])
inputs

array([[[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]],

       [[1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 0, 0]]])

In [3]:
tf.set_random_seed(35)
tf_inputs = tf.constant(inputs, dtype=tf.float32)
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=3)
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype=tf.float32, inputs=tf_inputs)

print('outputs:', outputs) # Tensor("rnn/transpose_1:0", shape=(2, 4, 3), dtype=float32)
print('state:', state) # Tensor("rnn/while/Exit_3:0", shape=(2, 3), dtype=float32)
print('weights')
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)
    # <tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
    # <tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
outputs: Tensor("rnn/transpose_1:0", shape=(2, 4, 3), dtype=float32)
state: Tensor("rnn/while/Exit_3:0", shape=(2, 3), dtype=float32)
weights
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


In [14]:
variables_name = [v.name for v in tf.trainable_variables()]
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])
    print('outputs:\n', output_run, sep='')
    # 두 문장의 첫 단어 출력값이 [ 0.37247857  0.27281666 -0.03415733]로 같다.
    # 첫 단어의 출력값이 같은 이유는 이전 단어가 존재하지 않기 때문에 이전 상태값이 존재하지 않기 때문이다.
    # 두 번째 단어부터의 출력값은 두 문장이 [-0.12360744 -0.23688789 -0.07885218]와 [-0.62467164  0.70083857  0.17365289]처럼
    # 다르게 나타난다.
    # 두 번째 단어부터 출력값이 다른 이유는 이전 상태값이 현재 출력에 영향을 주기 때문이다.
    print('state:\n', state_run, sep='')
    # 상태값은 항상 마지막 단어의 출력값과 동일하다.
    # 첫 번째 단어 출력값
    # [[ 0.37247857  0.27281666 -0.03415733] # 이전 단어가 없기 때문에 같은 값이 출력된다.
    #  [-0.12360744 -0.23688789 -0.07885218]
    #  [-0.27765325 -0.14212623 -0.44862753]
    #  [-0.7634894  -0.13153659 -0.1189682 ]] # 첫 번째 문장의 마지막 단어 출력값 => 상태값
    # =========================================================================================
    # 두 번째 단어 출력값
    # [[ 0.37247857  0.27281666 -0.03415733] # 이전 단어가 없기 때문에 같은 값이 출력된다.
    #  [-0.62467164  0.70083857  0.17365289]
    #  [ 0.10053852  0.2770306   0.05076913]
    #  [-0.05543748 -0.37285638 -0.04752919]] # 두 번째 문장의 마지막 단어 출력값 => 상태값
    # =========================================================================================
    # 상태값
    # [-0.7634894  -0.13153659 -0.1189682 ] # 첫 번째 문장의 상태값은 첫 번째 문장의 마지막 단어 출력값과 같다.
    # [-0.05543748 -0.37285638 -0.04752919] # 두 번째 문장의 상태값은 두 번째 문장의 마지막 단어 출력값과 같다.
    print('weights & bias')
    values = sess.run(variables_name)
    for k, v in zip(variables_name, values):
        print('{}\n{}'.format(k, v))

outputs:
[[[ 0.37247857  0.27281666 -0.03415733]
  [-0.12360744 -0.23688789 -0.07885218]
  [-0.27765325 -0.14212623 -0.44862753]
  [-0.7634894  -0.13153659 -0.1189682 ]]

 [[ 0.37247857  0.27281666 -0.03415733]
  [-0.62467164  0.70083857  0.17365289]
  [ 0.10053852  0.2770306   0.05076913]
  [-0.05543748 -0.37285638 -0.04752919]]]
state:
[[-0.7634894  -0.13153659 -0.1189682 ]
 [-0.05543748 -0.37285638 -0.04752919]]
weights & bias
rnn/basic_rnn_cell/kernel:0
[[ 0.39129794  0.27990448 -0.03417063]
 [-0.13734835 -0.6881115  -0.19269955]
 [-0.19838947  0.15787274 -0.3503854 ]
 [-0.74573535  0.42230952  0.06174737]
 [-0.07638937  0.737661   -0.0250259 ]
 [ 0.22168279  0.6998607   0.48419607]
 [ 0.55390215  0.55799115  0.26617455]]
rnn/basic_rnn_cell/bias:0
[0. 0. 0.]
